## importing libraries


In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import urllib
from tqdm import tqdm
import wget

## assign standards, subjects, medium of languages

In [2]:
classical = ['201','202','203','204','205','206','207','208','209','210','211', '212'] #select one by one 
sub = ['401','402','403','407','408', '409', '410','411']
med = ['301','302','303','304','305','306','307','309','310','311']

In [6]:
classical = ['212']
sub = ['401','402','403','407','408','410','411']
med = ['301','302','303','304','305','306','307','309','310','311']

## parse through aspx based website

In [7]:
def fun(classy,subjects,mediums):

    dump = classy+" "+mediums+" "+subjects
    page_no = 1
    name_of_chk_include = [i for i in dump.split()]
    txt_selected = ' '.join(name_of_chk_include)
    event_argument = txt_selected + f'#{page_no}'

    cookies = {
        'ASP.NET_SessionId': '2untnv42gdmvl3sybtllh2jg',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'X-Requested-With': 'XMLHttpRequest',
        'X-MicrosoftAjax': 'Delta=true',
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
        'Origin': 'https://cart.ebalbharati.in',
        'Connection': 'keep-alive',
        'Referer': 'https://cart.ebalbharati.in/BalBooks/ebook.aspx',
    }

    data = {
    'ScriptManager1': 'upBtn|upBtn',
    'rdbYear': 'on',
    f'chk{name_of_chk_include[0]}': 'on',
    f'chk{name_of_chk_include[1]}': 'on',
    f'chk{name_of_chk_include[2]}': 'on',
    'txtSelected': txt_selected,
    'txtyear': '2020',
    '__EVENTTARGET': 'upBtn',
    '__EVENTARGUMENT': event_argument,
    '__VIEWSTATE': '...',
    '__VIEWSTATEGENERATOR': '4401DBE3',
    '__EVENTVALIDATION': '/wEdAARm1jOA0AVaSftsCwPtfAonabsYFsoySZpnEvNskTLBAVttzaFSJIojwn7zN9KXlw6lYi8iZJYQC8KFtG27vcpYSTKBcR1J/4KKL1hrorJutBbhtPAfHYzFjYPQwb/VngY=',
    '__ASYNCPOST': 'true'
    }

    response = requests.get('https://cart.ebalbharati.in/BalBooks/ebook.aspx', headers=headers, cookies=cookies, data=data)
    soup = BeautifulSoup(response.text, 'html.parser')
    data = soup.find_all('div',class_='button')
    links = []
    linkys = []

    for link in data:
        
        temp = link.get('onclick')
        temp = [temp]
        links.append(temp)

    for i in links:
        for j in i:
            dym = j.split('"')
            linkys.append(dym[1])

    return linkys

## get link for each standard

In [10]:
urls_links = []
for i in tqdm(classical):
    for j in sub:
        for k in med:
            d = fun(i,j,k)
            print(urls_links)
            urls_links.append(d)

In [1]:
# urls_links

## get the exact length of the each standard

In [127]:
jumpy = []
for kkk in range(0,len(urls_links),10):
    jumpy.append(urls_links[kkk:kkk+10])

In [2]:
print(len(jumpy))

In [3]:
jumpy

In [4]:
#i - > No of subject
#j - > No of Language list in particular subject
 
# for i in range(0,len(jumpy)):
#     for j in range(0,len(jumpy[i])):
#         print(i)
        

## download the exact file at the exact location

In [130]:
out_std = ['1','2','3','4','5','6','7','8','9','10','11','12']

out_lang =['ma','hi','en','ur','gu','ka','si','te','ta','be']

total_out_sub = ['lang','math','sci','his','geo','civ','evs','others']

out_sub = ['lang','math','sci','his','geo','evs','others']

medium = ['01','02','03','04','05','06','07','09','10','11']


for i in tqdm(range(0,len(jumpy))):
    for j in range(0,len(jumpy[i])):
        for k in range(0,len(jumpy[i][j])):
            url = jumpy[i][j][k]
            d1 = url.split("/")
            filename = d1[5]
            splitter = filename.split(".")
            splitter = splitter[0][2:4]
            medium_holder = medium[j]
            path = out_std[11]+'/'+out_sub[i]+'/'+out_lang[j]+'/'

            if not os.path.exists(path):
                    os.makedirs(path)
            if not os.path.exists(os.path.join(path, filename)):
                try:
                    if(medium_holder==splitter):
                        desti = os.path.join(path,filename)
                        wget.download(url, desti)
                except Exception as e:
                    print(e)
                    continue

100%|██████████| 7/7 [49:57<00:00, 428.25s/it]   
